In [1]:
import os
import pickle
import json
import xgboost
import train_xgb

from azureml.core import Workspace

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

quick-starts-ws-154414
aml-quickstarts-154414
southcentralus
cdbe0b43-92a0-4715-838a-f2648cc7ad21


In [3]:
with open('outputs/best_hdr_metrics.json', 'r') as f:
    best_run_metrics = json.load(f)
best_run_metrics

{'Learning rate': 0.061044644881138845,
 'Gamma': 7.077858161938429,
 'Maximum depth': 5.0,
 'r2_score': 0.9283421514369813}

In [4]:
! python train_xgb.py --learning_rate 0.061044644881138845 --gamma 7.077858161938429 --max_depth 5

X_train.shape = (1095, 79), X_test.shape = (365, 79)
Attempted to log scalar metric Learning rate:
0.061044644881138845
Attempted to log scalar metric Gamma:
7.077858161938429
Attempted to log scalar metric Maximum depth:
5.0
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \
Attempted to log scalar metric r2_score:
0.8649394875653554
Writting r2 score = 0.8649394875653554 into a log.


In [5]:
with open('outputs/model.pkl', 'rb') as file:
    xgb = pickle.load(file)
xgb

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=7.077858161938429,
             importance_type='gain', learning_rate=0.061044644881138845,
             max_delta_step=0, max_depth=5, min_child_weight=1, missing=nan,
             n_estimators=100, n_jobs=1, nthread=None,
             objective='reg:squarederror', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
             subsample=1, verbosity=1)

In [6]:
from azureml.core.model import Model

# Register the best model
model = Model.register(ws, model_path='outputs/model.pkl', model_name='Ames-Housing-XGB-Model', tags=best_run_metrics)
print(model.name, model.id, model.version, sep='\t')

Registering model Ames-Housing-XGB-Model
Ames-Housing-XGB-Model	Ames-Housing-XGB-Model:1	1


In [7]:
from azureml.core import Environment
from azureml.core.model import InferenceConfig

env = Environment(name="project_environment")
dummy_inference_config = InferenceConfig(
    environment=env,
    source_directory=".",
    entry_script="echo_score.py",
)

In [8]:
from azureml.core.webservice import LocalWebservice

deployment_config = LocalWebservice.deploy_configuration(port=6789)

In [9]:
service = Model.deploy(
    ws,
    "myservice",
    [model],
    dummy_inference_config,
    deployment_config,
    overwrite=True,
)
service.wait_for_deployment(show_output=True)

Generating Docker build context.
2021/08/13 07:40:32 Downloading source code...
2021/08/13 07:40:33 Finished downloading source code
2021/08/13 07:40:33 Creating Docker network: acb_default_network, driver: 'bridge'
2021/08/13 07:40:34 Successfully set up Docker network: acb_default_network
2021/08/13 07:40:34 Setting up Docker configuration...
2021/08/13 07:40:34 Successfully set up Docker configuration
2021/08/13 07:40:34 Logging in to registry: 6e98cd47afdf4a08b0da1984384c187c.azurecr.io
2021/08/13 07:40:35 Successfully logged into 6e98cd47afdf4a08b0da1984384c187c.azurecr.io
2021/08/13 07:40:35 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'
2021/08/13 07:40:35 Scanning for dependencies...
2021/08/13 07:40:36 Successfully scanned dependencies
2021/08/13 07:40:36 Launching container with name: acb_step_0
Sending build context to Docker daemon  66.56kB
Step 1/18 : FROM mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20210615.

Error: Container has crashed. Did your init method fail?



WebserviceException: WebserviceException:
	Message: Error: Container has crashed. Did your init method fail?
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Error: Container has crashed. Did your init method fail?"
    }
}

In [10]:
os.environ['AZUREML_MODEL_DIR']

KeyError: 'AZUREML_MODEL_DIR'

In [11]:
import requests
import json

uri = service.scoring_uri
requests.get("http://localhost:6789")
headers = {"Content-Type": "application/json"}
data = {
    "query": "What color is the fox",
    "context": "The quick brown fox jumped over the lazy dog.",
}
data = json.dumps(data)
response = requests.post(uri, data=data, headers=headers)
print(response.json())

ConnectionError: HTTPConnectionPool(host='localhost', port=6789): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7ff70d6bae80>: Failed to establish a new connection: [Errno 111] Connection refused',))